<a href="https://colab.research.google.com/github/veda0696-stack/DL-assignment/blob/main/CatDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
Optimized Cats vs Dogs CNN
"""

# -----------------------------
# 1. IMPORT LIBRARIES
# -----------------------------
import os
import zipfile
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization, Input
)
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# -----------------------------
# 2. EXTRACT DATASET
# -----------------------------
local_zip = '/content/cats_and_dogs_filtered.zip'
extract_path = '/content/cats_and_dogs_filtered'

# Check if the zip file exists before attempting to extract
if os.path.exists(local_zip):
    with zipfile.ZipFile(local_zip, 'r') as zip_ref:
        zip_ref.extractall('/content/') # Extract to content directory
    print(f"'{local_zip}' extracted successfully to '/content/'")
else:
    print(f"Error: Zip file not found at '{local_zip}'. Please upload the file.")
    # Optionally, you might want to stop execution or handle this more gracefully

# -----------------------------
# 3. DIRECTORY SETUP
# -----------------------------
base_dir = extract_path
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Check if directories exist before listing content
if os.path.exists(train_cats_dir):
    print("Training cats:", len(os.listdir(train_cats_dir)))
else:
    print(f"Training cats directory not found at {train_cats_dir}")

if os.path.exists(train_dogs_dir):
    print("Training dogs:", len(os.listdir(train_dogs_dir)))
else:
    print(f"Training dogs directory not found at {train_dogs_dir}")

if os.path.exists(validation_cats_dir):
    print("Validation cats:", len(os.listdir(validation_cats_dir)))
else:
    print(f"Validation cats directory not found at {validation_cats_dir}")

if os.path.exists(validation_dogs_dir):
    print("Validation dogs:", len(os.listdir(validation_dogs_dir)))
else:
    print(f"Validation dogs directory not found at {validation_dogs_dir}")

# -----------------------------
# 4. DATA GENERATORS (OPTIMIZED)
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Ensure directories exist before creating generators
if os.path.exists(train_dir) and os.path.exists(validation_dir):
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150,150),
        batch_size=32,
        class_mode='binary'
    )

    validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150,150),
        batch_size=32,
        class_mode='binary'
    )
else:
    print("Error: Training or validation directories not found. Cannot create data generators.")
    train_generator = None # Set to None to prevent further errors
    validation_generator = None # Set to None to prevent further errors

# -----------------------------
# 5. OPTIMIZED CNN MODEL
# -----------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

# -----------------------------
# 6. COMPILE MODEL
# -----------------------------
model.compile(
    optimizer=RMSprop(learning_rate=0.0005),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 7. TRAIN MODEL
# -----------------------------
if train_generator and validation_generator:
    history = model.fit(
        train_generator,
        epochs=15,
        validation_data=validation_generator,
        verbose=2
    )
else:
    print("Skipping model training due to missing data generators.")
    history = None # Set history to None if training is skipped

# -----------------------------
# 8. PLOT ACCURACY & LOSS
# -----------------------------
if history:
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.figure()
    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')
    plt.show()

    plt.figure()
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.legend()
    plt.title('Loss')
    plt.show()
else:
    print("Skipping accuracy and loss plotting because model was not trained.")

# -----------------------------
# 9. IMAGE PREDICTION
# -----------------------------
def predict_image(img_path):
    if not os.path.exists(img_path):
        print(f"Error: Image file not found at '{img_path}'.")
        return

    img = load_img(img_path, target_size=(150,150))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    prediction = model.predict(x)[0][0]
    print("Prediction value:", prediction)

    if prediction > 0.5:
        print("➡ DOG")
    else:
        print("➡ CAT")

# Example prediction - Update this path if you have a test image
# For example, after extracting the zip, you might have images in /content/cats_and_dogs_filtered/validation/dogs/dog.100.jpg
# Ensure 'MyRoundDog.jpg' is uploaded to the Colab environment or adjust the path accordingly.
predict_image('/content/MyRoundDog.jpg') # Changed path from E:\DL\MyRoundDog.jpg

# -----------------------------
# 10. FEATURE MAP VISUALIZATION
# -----------------------------
if train_cats_dir and train_dogs_dir and os.path.exists(train_cats_dir) and os.path.exists(train_dogs_dir):
    layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name.lower()]
    visual_model = Model(inputs=model.input, outputs=layer_outputs)

    # Ensure there are images to choose from before trying to list directories
    all_image_paths = []
    if os.path.exists(train_cats_dir):
        all_image_paths.extend([os.path.join(train_cats_dir, f) for f in os.listdir(train_cats_dir)])
    if os.path.exists(train_dogs_dir):
        all_image_paths.extend([os.path.join(train_dogs_dir, f) for f in os.listdir(train_dogs_dir)])

    if all_image_paths:
        img_path = random.choice(all_image_paths)

        img = load_img(img_path, target_size=(150,150))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x /= 255.0

        feature_maps = visual_model.predict(x)

        for fmap in feature_maps:
            plt.figure(figsize=(12,3))
            for i in range(min(6, fmap.shape[-1])):
                plt.subplot(1,6,i+1)
                plt.imshow(fmap[0,:,:,i], cmap='viridis')
                plt.axis('off')
            plt.show()
    else:
        print("No images found in training directories for feature map visualization.")
else:
    print("Skipping feature map visualization due to missing training directories.")


Error: Zip file not found at '/content/cats_and_dogs_filtered.zip'. Please upload the file.
Training cats directory not found at /content/cats_and_dogs_filtered/train/cats
Training dogs directory not found at /content/cats_and_dogs_filtered/train/dogs
Validation cats directory not found at /content/cats_and_dogs_filtered/validation/cats
Validation dogs directory not found at /content/cats_and_dogs_filtered/validation/dogs
Error: Training or validation directories not found. Cannot create data generators.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,564,609 (36.49 MB)

 Trainable params: 9,564,161 (36.48 MB)

 Non-trainable params: 448 (1.75 KB)

Skipping model training due to missing data generators.
Skipping accuracy and loss plotting because model was not trained.
Error: Image file not found at '/content/MyRoundDog.jpg'.
Skipping feature map visualization due to missing training directories.
